In [1]:
# import dependencies
import os
import csv
from pandas import DataFrame
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import time
from sqlalchemy import create_engine

In [17]:
# read csv into data frame
citations = pd.read_csv('data/citations_db_ready/citations_0000000001.csv', nrows = 1000)
citations.head()

,agency,body_style,color,fine_amount,issue_date,issue_time,latitude,location,longitude,make,meter_id,route,ticket_number,violation_code,violation_description
0,2,PA,BK,58.0,2015-12-21,2055.0,-118.401151,200 WORLD WAY,33.945212,NISS,NaN,2R2,1104803000,8939,WHITE CURB
1,2,PA,WH,NaN,2015-12-26,1515.0,-118.401009,100 WORLD WAY,33.945212,ACUR,NaN,2F11,1104820732,000,17104h
2,1,PA,BK,93.0,2015-12-21,1041.0,-118.363082,10635 FRIAR ST,34.185788,HOND,NaN,15EL5,1108321723,8056E4,RED ZONE
3,1,NaN,GO,363.0,2015-12-19,1550.0,-118.438092,14006 RIVERSIDE DR,34.157607,HOND,NaN,35MQ1,1109139006,225078A,HANDICAP/NO DP ID
4,36,PA,WH,68.0,2015-12-26,1815.0,-118.311225,3200 CANYON DR,34.126932,TOYO,NaN,NaN,1109455266,8603,PK IN PROH AREA


### db transfer temporary process

In [3]:
citations_2 = citations.fillna(0)

In [18]:
# preview the missing values and the % of missing values in each column
df = citations_2
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_values_table(df)

Selected dataframe has 15 columns.
There are 8 columns that have missing values.


,Missing Values,% of Total Values
meter_id,1071143,67.6
route,6144,0.4
make,1069,0.1
fine_amount,863,0.1
body_style,822,0.1
color,373,0.0
issue_time,266,0.0
violation_description,95,0.0


In [5]:
citations_2.drop(["issue_time", "meter_id", "route"], axis = 1, inplace = True)

In [6]:
citations_2['issue_date'] = pd.to_datetime(citations_2['issue_date'], dayfirst=True)

In [16]:
# check data type in dataframe convert data to an appropriate format
citations.dtypes

agency                     int64
body_style                object
color                     object
fine_amount              float64
issue_date                object
issue_time               float64
latitude                 float64
location                  object
longitude                float64
make                      object
meter_id                  object
route                     object
ticket_number              int64
violation_code            object
violation_description     object
dtype: object

In [11]:
citations['issue_time'] = pd.to_datetime(citations['issue_time'])

In [15]:
citations['issue_time'] = pd.to_datetime(citations['issue_time'],format ='%H:%M:%S')

ValueError: time data 2055 does not match format '%H:%M:%S' (match)

In [13]:
citations

,agency,body_style,color,fine_amount,issue_date,issue_time,latitude,location,longitude,make,meter_id,route,ticket_number,violation_code,violation_description
0,2,PA,BK,58.0,2015-12-21,00:00:00.000002,-118.401151,200 WORLD WAY,33.945212,NISS,NaN,2R2,1104803000,8939,WHITE CURB
1,2,PA,WH,NaN,2015-12-26,00:00:00.000001,-118.401009,100 WORLD WAY,33.945212,ACUR,NaN,2F11,1104820732,000,17104h
2,1,PA,BK,93.0,2015-12-21,00:00:00.000001,-118.363082,10635 FRIAR ST,34.185788,HOND,NaN,15EL5,1108321723,8056E4,RED ZONE
3,1,NaN,GO,363.0,2015-12-19,00:00:00.000001,-118.438092,14006 RIVERSIDE DR,34.157607,HOND,NaN,35MQ1,1109139006,225078A,HANDICAP/NO DP ID
4,36,PA,WH,68.0,2015-12-26,00:00:00.000001,-118.311225,3200 CANYON DR,34.126932,TOYO,NaN,NaN,1109455266,8603,PK IN PROH AREA
5,36,PA,GY,68.0,2015-12-27,00:00:00.000001,-118.311225,3200 CANYON DR,34.126932,INFI,NaN,NaN,1109455270,8603,PK IN PROH AREA
6,36,TR,BL,68.0,2015-12-23,00:00:00.000002,-118.235839,835 N ACADEMY RD,34.079291,TOYO,NaN,NaN,1109455745,8603,PK IN PROH AREA
7,36,PA,GY,68.0,2015-12-26,00:00:00.000002,-118.283476,3787 MARATHON,34.083872,TOYO,NaN,NaN,1109455970,8603,PK IN PROH AREA
8,36,PA,WH,68.0,2015-12-23,00:00:00.000002,-118.408075,2551 MOTOR,34.046872,MERC,NaN,NaN,1109810450,8603,PK IN PROH AREA
9,36,SU,BK,68.0,2015-12-26,00:00:00.000002,-118.189051,1141 EAGLE VISTA DR,34.143399,SCIO,NaN,NaN,1109811360,8603,PK IN PROH AREA


In [14]:
# Create database connection
connection_string = "root:root@127.0.0.1:3306/la_parking_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [15]:
# Confirm tables
engine.table_names()

['meters_inventory', 'parking_citations']

In [16]:
# Load DataFrames into database
citations_2.to_sql(name = 'parking_citations', con = engine, if_exists = 'append', index = False) 

DataError: (pymysql.err.DataError) (1265, "Data truncated for column 'ticket_number' at row 1388")
[SQL: INSERT INTO parking_citations (agency, body_style, color, fine_amount, issue_date, latitude, location, longitude, make, ticket_number, violation_code, violation_description) VALUES (%(agency)s, %(body_style)s, %(color)s, %(fine_amount)s, %(issue_date)s, %(latitude)s, %(location)s, %(longitude)s, %(make)s, %(ticket_number)s, %(violation_code)s, %(violation_description)s)]
[parameters: ({'agency': 2, 'body_style': 'PA', 'color': 'BK', 'fine_amount': 58.0, 'issue_date': datetime.datetime(2015, 12, 21, 0, 0), 'latitude': -118.40115138698648, 'location': '200 WORLD WAY', 'longitude': 33.94521197960505, 'make': 'NISS', 'ticket_number': 1104803000, 'violation_code': '8939', 'violation_description': 'WHITE CURB'}, {'agency': 2, 'body_style': 'PA', 'color': 'WH', 'fine_amount': 0.0, 'issue_date': datetime.datetime(2015, 12, 26, 0, 0), 'latitude': -118.40100895210728, 'location': '100 WORLD W